In [ ]:
from sklearn import preprocessing
import pandas as pd


### load data

In [ ]:
cifar = pd.read_csv('CIFAR-10.csv',header=None)
columns_names = [f"p{i}" for i in range(cifar.shape[1]-1)]
columns_names.append('label')
cifar.columns = columns_names
cifar

### normalization

In [ ]:
y = cifar['label']
x = cifar.drop(['label'],axis=1)/255

mean__ = x.mean(axis=0)
std__ = y.std(axis=0)

x = (x - mean__) / std__
x

### mlp archi

### split dataset (20% validation)

### mini-batch SGD (batch_size = 128, learning_rate = 0.01, epochs = 50)

### learning curves on train/validation set

### L2 norm regularization

### learning curves on train/validation set

### comparaison

### dropout layer (rates 0.2, 0.3, 0.5)

### comparaison

### early stopping technique

### batch normalization layer

### random search to find best hyperparameter